* target은 d+1에 대한 d+2일의 변화량 (target(d) = (close(d+2)-close(d+1))/close(d+1)*100)
* test_x는 날짜별로 2000개의 종목이 한 iter에 들어옴

In [1]:
import numpy as np 
import pandas as pd

import os
import datetime
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jpx-tokyo-stock-exchange-prediction/stock_list.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/sample_submission.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/options.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/financials.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/secondary_stock_prices.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/trades.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/stock_prices.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/jpx_tokyo_market_prediction/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jpx-tokyo-stock-exchange-prediction/jpx_tokyo_market_prediction/__init__.py
/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/stock_fin_spec.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/trades_spec.csv
/kaggle/input/jpx-tokyo-stock-

# data load and split

In [2]:
# data load
train_folder = "/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/"
test_folder = "/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/"

train_df = pd.read_csv(train_folder+"/stock_prices.csv")
test_df = pd.read_csv(test_folder+"/stock_prices.csv")

# data split
train_y = train_df[["Target"]]
train_x = train_df.drop(["Target"],axis=1)

display(train_x,train_y)

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False


,Target
0,0.000730
1,0.012324
2,0.006154
3,0.011053
4,0.003026
...,...
2332526,0.034816
2332527,0.025478
2332528,-0.004302
2332529,0.009098


# funtions

In [3]:
# def str_to_time(str_date):
#     return datetime.datetime.strptime(str_date,"%Y-%m-%d")

def str_to_time(df):
    def str_to_year(str_date):
        return int(str_date.split("-")[0])

    def str_to_month(str_date):
        return int(str_date.split("-")[1])

    def str_to_day(str_date):
        return int(str_date.split("-")[2])

    df["year"] = df["Date"].apply(str_to_year)
    df["month"] = df["Date"].apply(str_to_month)
    df["day"] = df["Date"].apply(str_to_day)
    
    df.drop(["Date"],axis=1,inplace=True)

In [4]:
train_x.drop(["RowId"],axis=1,inplace=True)
str_to_time(train_x) # make year, month, day column
train_x["ExpectedDividend"].fillna(0,inplace=True) # make 0 if ExpedtedDividend Nan
train_x

,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,year,month,day
0,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,0.0,False,2017,1,4
1,1332,568.0,576.0,563.0,571.0,2798500,1.0,0.0,False,2017,1,4
2,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,0.0,False,2017,1,4
3,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,0.0,False,2017,1,4
4,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,0.0,False,2017,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,9990,514.0,528.0,513.0,528.0,44200,1.0,0.0,False,2021,12,3
2332527,9991,782.0,794.0,782.0,794.0,35900,1.0,0.0,False,2021,12,3
2332528,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,0.0,False,2021,12,3
2332529,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,0.0,False,2021,12,3


# 모델링